In [ ]:
# with thanks to Richard Chadwick, https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25
# much of the code for this scraper is either taken or adapted from his

In [ ]:
# !pip install tweepy

In [4]:
import tweepy as tw
import datetime
import pandas as pd

In [5]:
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        'xOM5aibCz665Rz3Bq6hiOlXD0',
        'consumer_secret':     '9F5KTULDv9YkJY8jJXW4g735toRziLW13ZTWV7l33h3LAPDF4F',
        'access_token_key':    '20414579-PwO6nnfmFWfIvC4QNWvQMK25H63P8b61TINjby3Hj',
        'access_token_secret': 'pOJpLVA7yzSdSY4JBqytwwU4Pi28m0IAmw38KnCtiP8m7'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tw.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tw.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_retweets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [6]:
miner=TweetMiner(result_limit = 200 )

In [7]:
import time

handle_list = ['list of handles you want the timelines of']

twitter_dict = {}
counter = 0

for name in handle_list:
    try:
        twitter_dict[name] = []
        twitter_dict[name].append(
            miner.mine_user_tweets(user=name, max_pages=17))
        counter = counter + 1
        if counter % 40 == 0:
            time.sleep(900)  # 15 minute sleep time
    # if name invalid print name and remove key
    except:
        print(name, 'is invalid or locked')
        twitter_dict.pop(name)

all_tweets = pd.concat([pd.DataFrame(twitter_dict[i][0])
                        for i in twitter_dict])

In [8]:
mined_tweets = miner.mine_user_tweets(user='AZGOP', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [10]:
mined_tweets_df.head()

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
0,1366531294763474945,Arizona Republican Party,AZGOP,39,"Today, Chairwoman Kelli Ward comments on Presi...",2021-03-01 21:20:45.403762,2021-03-01 23:30:24,201,[],22075,None,Twitter Media Studio,None,None,None
1,1366365545960890372,Arizona Republican Party,AZGOP,64,True story. https://t.co/p3RkEGBRqE,2021-03-01 21:20:45.403762,2021-03-01 12:31:46,389,[],22075,None,Twitter for iPhone,None,None,None
2,1366247085746515972,Arizona Republican Party,AZGOP,369,Happen to catch President Trump's speech at CP...,2021-03-01 21:20:45.403762,2021-03-01 04:41:03,2657,[],22075,None,Twitter Web App,None,None,None
3,1365681877743329293,Arizona Republican Party,AZGOP,0,@FryHangar @sococro https://t.co/F8nWAtiH0W,2021-03-01 21:20:45.403762,2021-02-27 15:15:07,1,[],22075,None,Twitter for iPhone,None,None,None
4,1365560237751603205,Arizona Republican Party,AZGOP,0,@sococro We are leading the country.,2021-03-01 21:20:45.403762,2021-02-27 07:11:46,0,[],22075,None,Twitter for iPhone,None,None,None


In [11]:
azgop_tweets = mined_tweets_df

In [12]:
azgop_tweets.index

RangeIndex(start=0, stop=2866, step=1)

In [14]:
azgop_tweets.set_index('created_at', inplace=True)

In [15]:
azgop_tweets.sort_index(ascending=True, inplace=True)

In [16]:
azgop_tweets.head()

,tweet_id,name,screen_name,retweet_count,text,mined_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
created_at,,,,,,,,,,,,,,
2020-10-28 23:27:25,1321594448652902401,Arizona Republican Party,AZGOP,1477,RT @DanScavino: https://t.co/kYfjpWYjCK,2021-03-01 21:20:58.069814,0,[],22075,None,Twitter for iPhone,https://t.co/kYfjpWYjCK,None,None
2020-10-28 23:27:36,1321594494353956864,Arizona Republican Party,AZGOP,16,RT @jeremiahcota: Thank you POTUS for one last...,2021-03-01 21:20:58.069814,0,"[{'text': 'FourMoreYears', 'indices': [75, 89]...",22075,None,Twitter for iPhone,Thank you POTUS for one last rally!\n\nArizona...,None,None
2020-10-28 23:27:43,1321594523277877249,Arizona Republican Party,AZGOP,143,RT @TrumpStudents: LATINOS LOVE @REALDONALDTRU...,2021-03-01 21:20:58.069814,0,[],22075,None,Twitter for iPhone,LATINOS LOVE @REALDONALDTRUMP! https://t.co/is...,None,None
2020-10-28 23:27:50,1321594553598500864,Arizona Republican Party,AZGOP,548,"RT @GOP: “Today, I’m announcing the American D...",2021-03-01 21:20:58.069814,0,[],22075,None,Twitter for iPhone,"“Today, I’m announcing the American Dream Plan...",None,None
2020-10-28 23:28:03,1321594610821459968,Arizona Republican Party,AZGOP,2439,RT @GOPChairwoman: ARIZONA is fired up and rea...,2021-03-01 21:20:58.069814,0,[],22075,None,Twitter for iPhone,ARIZONA is fired up and ready to re-elect for ...,None,None


In [18]:
azgop_tweets.shape

(2866, 14)

In [ ]:
azgop_tweets.to_csv('./AZGOP1.csv')